In [1]:
import pathlib
import numpy as np
from fytok.Tokamak import Tokamak
from spdm.view import View as sp_view

input_path = pathlib.Path("/home/salmon/workspace/fytok_data/data/15MA inductive - burn")
output_path = f"/home/salmon/workspace/output"

2023-11-04 21:13:04,185 [   fytok]     INFO: 
#######################################################################################################################
    ______      _____     _
   / ____/_  __|_   _|__ | | __
  / /_  / / / /  | |/ _ \| |/ /
 / __/ / /_/ /   | | (_) |   <
/_/    \__, /    |_|\___/|_|\_\
      /____/

 Copyright (c) 2021-present Zhi YU (Institute of Plasma Physics Chinese Academy of Sciences)
            
 url: https://gitee.com/openfusion/fytok_tutorial 
      https://github.com/fusion-yun/fytok_tutorial

 ontology = imas/3, version = 0.2.1 ( extension=0.2.2.dev11+g706f667, imas_version=v3_38_1_dirty ) 

 Run by salmon on Surface at 2023-11-04T21:13:04.185654

#######################################################################################################################



In [2]:

tokamak = Tokamak(f"file+iterprofiles://{next(input_path.glob('*.xls')).as_posix()}",
                  f"file+geqdsk://{next(input_path.glob('**/*.txt')).as_posix()}",
                  device="iter",
                  transport_solver={"code":  {"name": "fytrans",
                                              "parameters": {
                                                  "rho_tor_norm": np.linspace(0.01, 0.995, 128),
                                                  "bvp_rms_mask": [0.96],
                                                  "hyper_diff": 0.0001
                                              }}})

tokamak.transport_solver.refresh(
    equilibrium=tokamak.equilibrium,
    core_profiles=tokamak.core_profiles,
    core_transport=tokamak.core_transport,
    core_sources=tokamak.core_sources,
)

2023-11-04 21:13:04,302 [   fytok]     INFO: Load module spdm.plugins.data.plugin_xml


2023-11-04 21:13:04,610 [   fytok]     INFO: Load module spdm.plugins.data.plugin_iterprofiles
2023-11-04 21:13:04,742 [   fytok]     INFO: Load module spdm.plugins.data.plugin_geqdsk
2023-11-04 21:13:04,762 [   fytok]     INFO: Load module fytok.plugins.transport_solver_numerics.fytrans
2023-11-04 21:13:05,108 [   fytok]     INFO: Load module fytok.plugins.equilibrium.eq_analyze


ValueError: need at least one array to stack

In [ ]:
solver_1d = tokamak.transport_solver.time_slice.current.solver_1d
equ = solver_1d.equation[0]
a, b, c, d, e, f, g,*_ = equ.coefficient
x = solver_1d.grid.rho_tor_norm
y = equ.primary_quantity.profile
yp = equ.primary_quantity.d_dr
flux = equ.primary_quantity.flux
dflux = equ.primary_quantity.dflux_dr
fig = sp_view.plot([
    # * equ.coefficient,
    [
        (d*yp, "$D$"),
        (e*y,   "$v$"),
        # (flux,   f"$\Gamma$"),
        ((c*(f - g * y)).I, r"$S$"),  # $\int c*(f - g \cdot y)$
        # ((-d*yp+e*y).I, r"$\Gamma$"),
        (-d*yp+e*y-(c*(f - g * y)).I, "residual"),

    ],
    y, yp

    # [

    #     (dflux, {"label": r"$d\Gamma$"}),
    #     # (Function(flux, x).d(), r"$d\Gamma$"),
    #     ((c*(f - g * y)), r"$c*(f - g \cdot y)$"),
    # ],
],
    x_axis=x)

In [ ]:
fig = sp_view.plot(tokamak.core_profiles.time_slice.current.profiles_1d.electrons.density_thermal,
                   x_axis=tokamak.core_profiles.time_slice.current.profiles_1d.grid.rho_tor_norm)

In [ ]:
J=debug_info[0].todense()

In [ ]:
J